# Indices and runtimes

* 15th of March 2018

``<jeep@cphbusiness.dk>``

# Agenda

* Normal form examples
* Exercise walk through
* Indices
  * Hash trees, B-trees R-trees, GIST trees and Bloom filters
* Joins
  * Inner joins, outer joins and cross joins
* Denormalisation
* Views
  * Logical views
  * Materialised views
* Hand-in

# Learning objectives
## Knowledge
The student must have knowledge of:

 * Various database types and the underlying models
 * A specific database system’s storage organisation  and query execution
 * A specific database system’s optimisation possibilities – including advantages and disadvantages
 * Database-specific security problems and their solutions
 * Concepts and issues when handling big data
 * The particular issues raised by having many simultaneous transactions, including in connection with distributed databases
 * Relational algebra (including its relationship to execution plans)

## Skills
The student can:

 * Transform logical data models into physical models in various database types
 * Implement database optimisation
 * Use parts of the administration tool to assist in the optimisation and tuning of existing databases, including the incorporation of a specific DBMS’ execution plans
 * Use a specific database system’s tools for handling simultaneous transactions
 * Use the programming and other facilities provided by a modern DBMS


## Competencies
The student can:
 
 * Analyse the application domain in order to select a database type
 * Divide responsibility for tasks between the application and DBMS during system development, to ensure the best possible implementation.


In [2]:
%load_ext sql
%sql postgresql://appdev@0.0.0.0/appdev

/home/jens/.virtualenvs/slides/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


'Connected: appdev@appdev'


# Normal form examples

* Stolen from 
  * [William Kent: A Simple Guide to Five Normal Forms in Relational Database Theory](http://www.bkent.net/Doc/simple5.htm)
  * [Computer science course Toronto 343](http://www.cs.toronto.edu/~faye/343/f07/lectures/wk12/wk12_BCNF2-up.pdf)
  
* Other nice examples
  * [Wikipedia](https://en.wikipedia.org/wiki/Database_normalization) (seriously)
  * [Rules of normalisation](https://web.archive.org/web/20080805014412/http://www.datamodel.org/NormalizationRules.html#bcnf)

## Boyce-Codd normal form, or normal form 3.5

**Definition**: No redundant functional dependencies. Or every determinant is a candidate key.

1. $X \rightarrow Y$ is a trivial functional dependency
2. $X$ is a *superkey*

<quote>Typically, any relation that is in 3NF is also in BCNF. However, a 3NF relation won't be in BCNF if (a) there are multiple candidate keys, (b) the keys are composed of multiple attributes, and (c) there are common attributes between the keys.</quote> - [Rules of data normalisation](https://web.archive.org/web/20080805014412/http://www.datamodel.org/NormalizationRules.html#bcnf)

**Motivation**: Less redundancy

## BCNF normal form examples

* ``Person(CPR, Name, Address)``
  * Is in BCNF, because CPR $\rightarrow$ (Name, Address)
  * Note that Name does not imply Address: it is *not* a transitive dependency


* ``HasAccount(AccountNumber, ClientId, OfficeId)``
  * The functional dependency AccountNumber $\rightarrow$ ClientId is not in BCNF
  * Because we have 1) multiple keys ($\{AccountNumber, ClientId\}$ and $\{ClientId, OfficeId\}$) b) multible attributes and c) keys that share attributes

* ``ManagerBranch(Manager, Project, Branch)``
  * Is in 3NF: no transitive dependencies
  * Is not in BCNF because each manager works in a particular branch ($Manager \rightarrow Branch$), but the branch can have different manager for different projects ($\{Project,Branch\} \rightarrow Manager$)

## Normal form 4

* Introduced in 1977 by Ronald Fagin
* Normally implies normal form 5

* **Definition**: No multivalued dependencies
  * For non-trivial multivalued dependencies X ↠ Y, X is a superkey

**Motivation**: De-duplication, by having fewer fields in the candidate keys

## Normal form 4 example

    -------------------------------
    | EMPLOYEE | SKILL | LANGUAGE |
    ===============================

*Problem*: A record should not contain two or more independent multi-valued records

*Solution*: Break up the table, and reduce the amount of columns in the candidate key
    
    --------------------  -----------------------
    | EMPLOYEE | SKILL |  | EMPLOYEE | LANGUAGE |
    ====================  =======================

## Normal form 5

* Also known as project-join normal form; Ronald Fagin 1979
* **Definition**: Every non-trivial join dependency in a table is implied by the candidate keys


**Motivation**: Avoid redundancy

<quote>Roughly speaking, we may say that a record type is in fifth normal form when its information content cannot be reconstructed from several smaller record types</quote> - William Kent

# Normal form 5 example

Suppose that agents represents companies and companies sells cars

    -----------------------------
    | AGENT | COMPANY | PRODUCT |
    |-------+---------+---------|
    | Smith | Ford    | car     | 
    | Smith | GM      | truck   | 
    -----------------------------

Now suppose that an agent sells a product, and represents a company throught that product. What is the problem with the above?

The candidate key doesn't imply all the columns!    
    
    -------------------   ---------------------   ------------------- 
    | AGENT | COMPANY |   | COMPANY | PRODUCT |   | AGENT | PRODUCT |
    |-------+---------|   |---------+---------|   |-------+---------|
    | Smith | Ford    |   | Ford    | car     |   | Smith | car     |
    | Smith | GM      |   | Ford    | truck   |   | Smith | truck   |
    | Jones | Ford    |   | GM      | car     |   | Jones | car     |
    -------------------   | GM      | truck   |   -------------------
                          ---------------------

## Exercise normalisation walk through

* This exercise is stolen from Mastering PostgreSQL by D. Fontaine

* There's neither a unique constraint nor primary key, so there is nothing preventing insertion of duplicates entries, violating 1NF.


* Some non-key attributes are not dependent on the key because we mix data from the Twitter account posting the  message and themessage itself, violating 2NF. This is the case with all the user's attributes, such as the nickname, bio, picture, followers, following, and listed attributes.


* We have transitive dependencies in the model, which violates 3NF.
  * The country and place attributes  depend  on  the location attribute. 
  * The hour attributes depend on the date attribute, as the hour alone can't represent when the tweet was transmitted.
  * The longitude and latitude should really be a single location column, given PostgreSQL's ability to deal with geometric data types. 

# Indexing

Indices are basically datastructures added to your relations. They give us:

* Fast lookups
* Checks on relation constraints

## Memory versus processing tradeoff

* Imagine a linked list of 1'000'000 elements
  * How much space does it take up?
  * How much work would it take in terms of memory and processing to find a particular element?

* Imagine a tree structure containing 1'000'000 elements
  * How much space does it take up?
  * How much work would it take in terms of memory and processing to find a particular element?

## The ``EXPLAIN`` query

* RDBMS have seriously advanced query execution planners
* The execution planners plan how your queries are executed
  * Are are queries performed correct?
  * How long will they take?
  * Can they be improved?
  
* ``EXPLAIN ANALYZE`` runs the query and analyses the result

In [ ]:
%sql EXPLAIN SELECT * FROM public.tweet;

In [ ]:
%sql EXPLAIN SELECT * FROM public.tweet WHERE uname = 'test';

## Hash trees

* Calculates a hash for your data
* Allows for faster comparisons

## B trees

* Extension of the binary tree, where you can have more than two leaves
* Default indices in PostgreSQL
* $O(log(n))$ lookup

![B tree](https://upload.wikimedia.org/wikipedia/commons/thumb/6/65/B-tree.svg/400px-B-tree.svg.png)

In [ ]:
%sql SELECT * from information_schema.columns where table_name = 'tweet';

In [ ]:
%sql select * from information_schema.tables where table_schema = 'pg_catalog';

In [ ]:
%sql select * from pg_indexes where tablename = 'tweet';

In [ ]:
%sql SELECT * FROM tweet WHERE id = 721318437075685382;

In [ ]:
%sql EXPLAIN ANALYZE SELECT * FROM tweet WHERE id = 721318437075685382;

In [ ]:
%sql ALTER TABLE tweet DROP CONSTRAINT tweet_pkey;

In [ ]:
%sql EXPLAIN ANALYZE SELECT * FROM tweet WHERE id = 721318437075685382;

In [ ]:
%sql CREATE UNIQUE INDEX tweet_pkey ON tweet USING btree (id)

# R-trees

* What happens when you have geospatial data?
* R-trees uses rectangles to index your data
* $O(log_M(n))$, where $_M$ is the amount of children per node

![R-tree](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6f/R-tree.svg/400px-R-tree.svg.png)

## BRIN indexes

**B**lock **R**ange **IN**dex

* Allows us to index ordered elements in blocks
* Close to horizontal scaling 
* Useful when handling extremely large data

![BRIN index](https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/BRIN_index.svg/220px-BRIN_index.svg.png)

In [ ]:
%sql CREATE INDEX tweet_brin_date ON tweet USING brin(date);

In [ ]:
%sql EXPLAIN ANALYZE SELECT COUNT(*) FROM tweet WHERE date > '2016-04-14' AND date < '2016-04-16';

In [ ]:
%sql DROP INDEX tweet_brin_date;

In [ ]:
%sql EXPLAIN ANALYZE SELECT COUNT(*) FROM tweet WHERE date > '2016-04-14' AND date < '2016-04-16';

# Bloom filters

Probabilistic datastructure that allows to test whether an element is contained in a set

* Have $n$ elements
* Have $k$ hash functions that uniformely distributes hash values from 0 to $n$
* Build a bit map that can either contain or not contain a bit from 0 to $n$
* Useful to test whether something is a part of a set with many attributes

![Bloom filter](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ac/Bloom_filter.svg/360px-Bloom_filter.svg.png)

# GiST trees

**G**eneral**i**sed **S**earch **T**rees

* Allows us to generalise our search structures, so we can build them on whicheever type we choose
* Contains a balanced tree structure like we've seen above
  * But with <key, pointer> pairs instead of integers (like in B-trees)
  * Instead, each node represents some condition that is true for all nodes in that leaf
* Works for tons of datastructures!
  * Including spatial data!

In [ ]:
%sql SELECT * FROM pg_indexes WHERE tablename = 'circuits';

In [ ]:
%sql SELECT * FROM circuits ORDER BY point(lng,lat) <-> point(2.349014, 48.864716);

In [ ]:
%sql EXPLAIN ANALYZE SELECT * FROM circuits ORDER BY position <-> point(2.349014, 48.864716);

In [ ]:
%sql DROP INDEX circuits_position_idx;

In [ ]:
%sql EXPLAIN ANALYZE SELECT * FROM circuits ORDER BY position <-> point(2.349014, 48.864716);

In [ ]:
%sql CREATE INDEX circuits_position_idx ON circuits USING gist ("position")

# Joins

* Joins calculates the *joint set* between two relations which fulfills a condition
  * Combines attributes from your relations into a single table

* Can be done in a number of ways
  * Equi join
  * Inner joins
  * Outer joins
  * Cross joins
  * Self joins

## Equi joins ($\theta$)

* Using equality ($=$) to identify where to join
* Example: 

      SELECT * FROM employee, department 
      WHERE employee.DepartmentID = department.DepartmentID;
      
      SELECT * FROM employee 
      JOIN customer 
      ON employee.employeeid = customer.supportrepid;

In [ ]:
%sql SELECT * FROM employee, customer WHERE employee.employeeid = customer.supportrepid;

In [ ]:
%sql EXPLAIN SELECT * FROM employee, customer WHERE employee.employeeid = customer.supportrepid;

In [ ]:
%sql SELECT * FROM employee JOIN customer ON employee.employeeid = customer.supportrepid;

In [ ]:
%sql EXPLAIN SELECT * FROM employee JOIN customer ON employee.employeeid = customer.supportrepid;

## Natural joins ($\bowtie$)

* Joins two relations where all their common attributes have the same values
  
$R \bowtie S = \left\{ r \cup s \ \vert \ r \in R \ \land \ s \in S \ \land \ \mathit{Fun}(r \cup s) \right\}$


Example:

      SELECT * FROM employee NATURAL JOIN department;


In [28]:
%sql SELECT * FROM employee NATURAL JOIN customer;

0 rows affected.


employeeid,lastname,firstname,address,city,state,country,postalcode,phone,fax,email,title,reportsto,birthdate,hiredate,customerid,company


## Inner joins

* Requires each row in the two joined tables to have matching column values

![Inner join](https://upload.wikimedia.org/wikipedia/commons/thumb/1/18/SQL_Join_-_07_A_Inner_Join_B.svg/220px-SQL_Join_-_07_A_Inner_Join_B.svg.png)

## Inner join example

    SELECT employee.LastName, employee.DepartmentID, department.DepartmentName 
    FROM employee 
    INNER JOIN department ON
    employee.DepartmentID = department.DepartmentID
    
* ``INNER`` is optional

In [ ]:
%%sql 
SELECT employee.employeeid, employee.lastname, customer.customerId, customer.lastname 
FROM employee 
INNER JOIN customer 
ON employee.employeeid = customer.supportrepid;

In [ ]:
%%sql 
EXPLAIN ANALYZE SELECT employee.employeeid, employee.lastname, customer.customerId, customer.lastname 
FROM employee 
INNER JOIN customer 
ON employee.employeeid = customer.supportrepid;

## Outer joins

Outer joins retains each row — even if no other matching row exists

* Left outer joins (⟕)
* Right outer joins (⟖)
* Full outer joins (⟗)

## Left outer joins

* Retains all the tuples in the left relations, *even if* the join condition didn't hold

Example

     SELECT *
     FROM employee 
     LEFT OUTER JOIN department ON employee.DepartmentID = department.DepartmentID;

![Left outer join](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f6/SQL_Join_-_01_A_Left_Join_B.svg/220px-SQL_Join_-_01_A_Left_Join_B.svg.png)

In [ ]:
%sql SELECT * FROM employee LEFT OUTER JOIN customer ON employee.employeeid = customer.supportrepid;

In [ ]:
%sql EXPLAIN SELECT * FROM employee LEFT OUTER JOIN customer ON employee.employeeid = customer.supportrepid;

## Right outer joins

* Retains all the tuples in the right relations, *even if* the join condition didn't hold

Example

     SELECT *
     FROM employee 
     RIGHT OUTER JOIN department ON employee.DepartmentID = department.DepartmentID;

![Right outer join](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/SQL_Join_-_03_A_Right_Join_B.svg/220px-SQL_Join_-_03_A_Right_Join_B.svg.png)

In [ ]:
%sql SELECT * FROM employee RIGHT OUTER JOIN customer ON employee.employeeid = customer.supportrepid;

In [ ]:
%sql EXPLAIN SELECT * FROM employee LEFT OUTER JOIN customer ON employee.employeeid = customer.supportrepid;

## Full outer join

Combines the results of a *left* and *right* outer join, returning null whenever a values is missing

Example

    SELECT *
    FROM employee FULL OUTER JOIN department
      ON employee.DepartmentID = department.DepartmentID;
      
![Full outer join](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3d/SQL_Join_-_05b_A_Full_Join_B.svg/220px-SQL_Join_-_05b_A_Full_Join_B.svg.png)

In [ ]:
%sql SELECT * FROM employee FULL OUTER JOIN customer ON employee.employeeid = customer.supportrepid;

In [ ]:
%sql EXPLAIN SELECT * FROM employee FULL OUTER JOIN customer ON employee.employeeid = customer.supportrepid;

## Formula one examples

In [ ]:
%sql SELECT date,name,results.time FROM races JOIN results on results.raceid = races.raceid AND results.position = 1;

In [ ]:
%sql EXPLAIN SELECT date,name,results.time FROM races JOIN results on results.raceid = races.raceid AND results.position = 1;

In [ ]:
%sql SELECT date,name,results.time FROM races LEFT JOIN results on results.raceid = races.raceid AND results.position = 1;

In [ ]:
%sql SELECT COUNT(*) FROM races RIGHT JOIN results on results.raceid = races.raceid AND results.position = 1;

In [ ]:
%sql SELECT COUNT(*) FROM races FULL JOIN results on results.raceid = races.raceid AND results.position = 1;

## Cross joins

The cartesian product of two relations

    SELECT *
    FROM employee CROSS JOIN department;

Equivalent to

    SELECT *
    FROM employee, department;


## Exercise

* Get the number of tuples in the ``CROSS JOIN`` between ``employee`` and ``customer``
* Now count the number of tuples in ``employee`` and ``customer``. How does that relate to the ``COUNT`` in the above cross join?


* Run ``EXPLAIN ANALYZE`` on the ``LEFT JOIN``, ``RIGHT JOIN`` and ``FULL JOIN``
* Explain what is happening
* Explain why they are different

## Multiple joins

You can join several table at once. Here I would like to get all the names of the winners:

    SELECT name, date, driver.surname FROM races 
    JOIN results ON results.raceid = races.raceid AND results.position = 1 
    JOIN drivers USING (driverid)
    ORDER BY date;

In [ ]:
%%sql
SELECT name, date, drivers.surname FROM races 
JOIN results ON results.raceid = races.raceid AND results.position = 1 
JOIN drivers USING (driverid)
ORDER BY date DESC;

## Join performance

* When you join several tables on several conditions, things quickly get very very slow
  * For every field you add, your complexity rises exponentially!
* Never forget indexing!

# Views

* A 'virtual' table, or a *stored* query
* Makes it possible to persist some of your queries



In [ ]:
%%sql EXPLAIN ANALYZE 
SELECT name, date, drivers.surname FROM races 
JOIN results ON results.raceid = races.raceid AND results.position = 1 
JOIN drivers USING (driverid)
ORDER BY date DESC;

In [ ]:
%%sql CREATE VIEW race_winners AS
SELECT name, date, drivers.surname FROM races 
JOIN results ON results.raceid = races.raceid AND results.position = 1 
JOIN drivers USING (driverid)
ORDER BY date DESC;

In [ ]:
%sql EXPLAIN ANALYZE SELECT * FROM race_winners;

## Materialised views

* In views, the view query is executed every time the view is queried
* Materialised views are cached views

      CREATE MATERIALIZED VIEW race_winners_cache AS race_winners;
      
* Materialised views are **static** snapshots
  * They will **NOT** update if your data is updated
  
      REFRESH MATERIALIZED VIEW race_winners_cache;

In [ ]:
%sql CREATE MATERIALIZED VIEW race_winners_cache AS SELECT * FROM race_winners;

In [111]:
%sql EXPLAIN ANALYZE SELECT * FROM race_winners_cache;

3 rows affected.


QUERY PLAN
Seq Scan on race_winners_cache (cost=0.00..10.70 rows=70 width=1036) (actual time=0.034..0.417 rows=974 loops=1)
Planning time: 0.212 ms
Execution time: 0.594 ms


# Hand-in: Materialised joins

* **Deadline**: 20th of March 12:00
* **Review deadline**: 21th of March 23:59

Your job is to 1) describe the content and function of an index, 2) analyse a join query on some tables in the f1db schema and 3) create a materialised view of it. You will have to

1. On the table ``circuits`` report:
  * What type of indices exists for the table and why they are of that type (and not some other type)
  * The amount of space each index takes up
2. We are talent scouts looking to win over some of the best new drivers there are. But we don't want them too old! Write a query that finds the winner of all the races, but only if they are younger than 38 years. The query should give return the date, driver surname, driver age, track time in milliseconds, race name and circuit name for all races.
3. Describe the query using ``EXPLAIN ANALYZE`` with at least 5 lines of text. Answer at least the following:
   * How many calls are you making? 
   * How long does it take to perform the query?
4. Create a materialized view of your query. Using ``EXPLAIN ANALYZE`` try to query the view. Write at least 5 lines of text explaining what's going on and why the query execution time changed.